# **PYTHON SCRIPT TO SEND EMAIL USING SMTP**
Sending a encrypted email through python can be done using **email package** ( library for managing email messages) and **smtplib** (more details below).

## **LIBRARIES,MODULES AND PACKAGES USED :**

**1.smtplib - SMTP Protocol Client :**

Smtplib ,an library or a module that defines an SMTP(Simple Mail Transfer protocol)client session object which can be used to send any email to any other internet machine with an SMTP or ESMTP listener daemon.

**2. ssl - Secure Socket Layer :**

Secure Socket Layer (SSL) provide security to the data that is transferred between web browser and server. SSL encrypt the link between a web server and a browser which ensures that all data passed between them remain private and free from attack.

**3.Email Package :**
The email package is a library for managing email messages, including MIME and other RFC 2822-based message documents. It is specifically not designed to do any sending of email messages to SMTP (RFC 2821), NNTP, or other servers; those are functions of modules such as smtplib and nntplib.

**4.MIME - Multipurpose Internet Mail Extensions :**

MIME(Multipurpose Internet Mail Extensions) is an Internet standard that extends the format of email messages to support text in character sets other than ASCII, as well as attachments of audio, video, images, and application programs.

 **MimeMultipart** type is one which represents a document that's comprised of multiple component parts, each of which may have its own individual MIME type.If a message has a multipart Content-Type, that means it consists of multiple messages and each of them defines its own Content-Type (which can again be multipart or something else). 

 **MimeBase** is  the base class for all the MIME-specific subclasses of Message. 

  **MIMEText** class is used to create MIME objects of major type text.
 The email package provides some convenient encodings in its encoders module. These encoders are actually used by the MIMEAudio and MIMEImage class constructors to provide default encodings.

**5. getpass - Portable Password input :**

The getpass module provides a portable way to handle such password prompts securely.





In [12]:
import os
import ssl
import smtplib
import getpass
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.multipart import MIMEBase
from email.mime.text import MIMEText
from email import encoders

**To create the message template, the external file path is used as argument to store message body**

In [13]:
def create_message_template(ext_message_file_path):
    message_template_file=open(ext_message_file_path,mode='w',encoding='utf-8')
    message_template_file.write(input("Enter your message:"))
    message_template_file.close()    

**The recipent details (recipents list) will be stored in external file path provided**

In [14]:
def create_recipents_list(ext_contacts_file_path):
    recipents=open(ext_contacts_file_path,mode='w',encoding='utf-8')
    rescipent_no=input("Enter the no. of rescipents:")
    for i in range(rescipent_no):
        recipent_list=input("Enter your Recipents address: ")
        recipents.write(recipents_list+'\n')
    recipents.close()

**To extract the list of recipents from external file,function extract_recipents_ext_file takes a ext_contacts_file_path as its argument. The data is split it into two lists recipents_name and email_id**

In [15]:
def extract_recipents_ext_file(ext_contacts_file_path):
    recipent_name=[]
    email_id=[]
    with open(ext_contacts_file_path, mode='r' , encoding ='utf-8') as ext_cont_file:
        for contact in ext_cont_file:
            recipent_name.append(contact.split()[0])
            email_id.append(contact.split()[1])
    return recipent_name,email_id

**The function extract_message_template extracts message from the external file path provided as argument as an object**

In [16]:
def extract_message_template(ext_message_file_path):
    with open(ext_message_file_path,'r',encoding='utf-8') as message_template_file:
        extracted_message=message_template_file.read()
    return extracted_message

 
To personalize the email for the recipent create the **MIMEMultipart message
object** and load it with appropriate headers for From, To, and Subject fields.And then attach the message body to the mail.

A **MIME attachment** with the content type **"application/octet-stream"** is a binary file. Typically, it will be an application or a document that must be opened in an application, such as a spreadsheet or word processor.

The payload is the part of transmitted data that is the actual intended message. **set_payload((attachment).read())** , Sets the entire message object’s payload to payload.

**Encoders** Encodes the payload into **base64** form and sets the Content-Transfer-Encoding header to base64. This is a good encoding to use when most of your payload is unprintable data since it is a more compact form than quoted-printable.

**add_header** extended header setting that adds a header to the message , additional header parameters can be provided as keyword arguments.

**Content-Disposition** is the header field to add and **attachment**  is the primary value for the header.
And attach the MIME with the **"application/octet-stream"** to the message.
Finally the personalized email is returned.


In [17]:
def send_personalized_email(user_Address,recipent_mail,extracted_message,subject,file_to_attach,attachment,filename):
    message=MIMEMultipart()
    message['From']=user_Address
    message['To']=recipent_mail
    message['Subject']=subject
    message_body=MIMEText(extracted_message,'plain')
    message.attach(message_body)
    attach_f=MIMEBase('application','octet-stream')
    attach_f.set_payload((attachment).read())
    encoders.encode_base64(attach_f)
    attach_f.add_header('Content-Disposition',"attachment; filename =%s" % filename)
    message.attach(attach_f)
    return message  

Import the smtplib and then create an **SMTP instance** that encapsulates an **SMTP connection**. It takes as parameter the host address and a port number, both of which entirely depends on the SMPT server settings of your particular email service provider.

**smtplib.SMTP(smtp_server, port)** is used to create an SMTP object.

**Extended HELO (EHLO)** is an  Extended Simple Mail Transfer Protocol (ESMTP) command sent by an email server to identify itself when connecting to another email server to start the process of sending an email.To identify yourself to the server, **.ehlo() ** should be called after creating an .**SMTP()** object, and again after **.starttls()**.

Instead of using **.SMTP_SSL**  to create a connection that is secure from the outset, we can create an unsecured SMTP connection and encrypt it using .**starttls().**

For each recipents_name and email_id in **zip(recipent_name,res_email_id)** send_personalized_email function is called to send personalized email to each recipent email_id.

In [18]:
def set_SMPT_server(smtp_server,port,user_Address,User_Password,ext_contacts_file_path,ext_message_file_path):
    subject=input("Enter Subject: ")
    #create_recipents_list(ext_contacts_file_path)
    create_message_template(ext_message_file_path)
    extracted_message=extract_message_template(ext_message_file_path)
    print(extracted_message)
    recipent_name,res_email_ids=extract_recipents_ext_file(ext_contacts_file_path)
    print(recipent_name,res_email_ids)
    context = ssl.create_default_context()
    file_to_attach=input("Enter the complete file path you wish to attach :")
    attachment=open(file_to_attach,"rb")
    filename=input("Rename the File: ")
    with smtplib.SMTP(smtp_server, port) as server:
        #server=server.smtplib.SMTP(host_Address,port)
        server.ehlo()
        server.starttls()
        server.ehlo()
        server.login(user_Address,User_Password)
        print("Congratulations!! Connection established!")
        recipent_details=zip(recipent_name,res_email_ids)
        for recipent_name,recipent_mail in recipent_details:
            message=send_personalized_email(user_Address,recipent_mail,extracted_message,subject,file_to_attach,attachment,filename)
            server.send_message(message)
            print("MESSAGE DELIVERED to {} !!".format(recipent_mail))
        server.quit()    

**Getting the working directory path and the credentials and passing the details as arguments to the set_SMPT_server function.SMTP host address(for gmail) is 'smtp.gmail.com'. Port number used for gmail is ‘587’.**

In [19]:
def main():
    directory_base_path=input("Enter the working directory path:")
    ext_contacts_file_name=input("Enter the filename containing recipents list:")
    ext_contacts_file_path=os.path.join(directory_base_path,ext_contacts_file_name)
    ext_message_file_name=input("Enter the filename containing the message :")
    ext_message_file_path=os.path.join(directory_base_path,ext_message_file_name)
    host_Address=input("Enter the smtp server Address:")
    port=int(input("Enter the port number:"))
    user_Address=input("Enter User Email Address:") 
    print("Enter User Password: ")
    User_Password=getpass.getpass()
    set_SMPT_server(host_Address,port,user_Address,User_Password,ext_contacts_file_path,ext_message_file_path)
    

In [20]:
 if __name__ == '__main__':
    try:
        main()
        print("THANK YOU. \n")
    except:
         print("Sorry!! Failed to send email!!")

Enter the working directory path:/home/mishadey/Desktop/MAJOR_PROJECT/CS_MAJOR_DRAFT
Enter the filename containing recipents list:ext_rescipents_file.txt
Enter the filename containing the message :message_template.txt
Enter the smtp server Address:smtp.gmail.com
Enter the port number:587
Enter User Email Address:misha.2june@gmail.com
Enter User Password: 
········
Enter Subject: CYBER SECURITY MAJOR PROJECT TEST MAIL 
Enter your message:THIS IS THE FINAL TEST MAIL.JUST CONFIRM IF YOU GET IT.
THIS IS THE FINAL TEST MAIL.JUST CONFIRM IF YOU GET IT.
['mishadey', 'mishelsakshi', 'ashwinrockonn123', 'rashmijalapally', 'singh.anshuman.singh8', 'tanishqm033', 'thanishvishal', 'theakshitkumar', 'uchiha72000', 'jaijoshi0310', 'mittalrohan2001', 'mohdanas1612', 'pandeynishantsagar'] ['misha.2june@gmail.com', 'mishelsakshi@gmail.com', 'ashwinrockon123@gmail.com', 'rashmijalapally@gmail.com', 'singh.anshuman.singh8@gmail.com', 'tanishqm033@gmail.com', 'thanishvishal@gmail.com', 'theakshitkumar@gma